In [1]:
import requests
import GEOparse
import xml.etree.ElementTree as ET
import pandas 
from pandas import DataFrame as DF
from dcicutils import ff_utils
from functions.notebook_functions import *

In [2]:
#Getting excel file
readExcel = pandas.read_excel(r'/Users/luisa/Documents/4DN/Wrangling/Hi-C Datasets.xlsx')
df = DF(readExcel, columns = ['order', 'title','link','submitter','score','grouping','famous','details'])


In [6]:
GEO_list = df['link']
scores_list = df['score']
pubmed_errors_list = []
parseGEO_error_list = []
organism_error_list = []
new_GEO_list = []

In [7]:
#New Table
pubID_col = []
Title_col = []
Authors_col = []
Human_col = []
Mouse_col = []
Keyword_col = []
year_col = []
status_col = []
link_col = []

In [8]:
for GEOacc in GEO_list:
    print(GEOacc)
    #Get the GEO information from GEO accession
    print("Get the GEO information from GEO accession")
    try:
        gse = GEOparse.get_GEO(GEOacc)
    except:
        parseGEO_list.append(GEOacc)
        pubID_col.append('N/A')
        Title_col.append('N/A')
        Authors_col.append('N/A')
        Keyword_col.append('N/A')
        year_col.append('N/A')
        Mouse_col.append('N/A')
        Human_col.append('N/A')
        continue
    #Get metadata
    metadata = gse.metadata
    #metadata.keys()
    #Get Data of interest
    try:
        pubID_i = gse.metadata['pubmed_id']
    except:
        pubID_col.append('N/A')
        Title_col.append('N/A')
        Authors_col.append('N/A')
        Keyword_col.append('N/A')
        year_col.append('N/A')
        Mouse_col.append('N/A')
        Human_col.append('N/A')
        pubmed_errors_list.append(GEOacc)
        continue
    pubID = str(pubID_i[0])
    pubID_col.append(pubID)
    
    #Getting infor from pubmed using the pubmed ID
    
    
    #Getting xml format
    pubmed_URL_xml = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=" + pubID +"&retmode=xml"
    #print(pubmed_URL_xml)
    response = requests.get(pubmed_URL_xml)
    with open('file.xml', 'wb') as f:
        f.write(response.content)
    tree = ET.parse('file.xml')
    
    #The Data is stored in the child of the root
    # child = tree.getchildren()
    # print(child)
    root = tree.getroot()
    for i in root:
        child = i
    
    #The data is stored in the child of the root
    #The info of interest is in the medlineCitation level
    medlineCitation = 'MedlineCitation'
    
    #This are the elements of interest in this datalevel:
    Article = 'Article'
    KeywordList = 'KeywordList' #each element only has a single item. no index needed with for loop
    
    #Elements of Interest in Article level:
    AuthorList = 'AuthorList'  # there are several, each item has last name index = 0, first name = 1, and abbrev = 2
    ArticleTitle = 'ArticleTitle' # there is only one object so index = 0
    ArticleDate = 'ArticleDate' # there are several, each item has a Year index = 0, Month index = 0, and day index = 3
    
    Author_List = []
    keyword_list = []
    
    for i in child:
        if medlineCitation in str(i):
            for item in i:
                if Article in str(item):
                    for ele in item:
                        if ArticleTitle in str(ele):
                            Article_Title = ele.text
                            Title_col.append(Article_Title)
                        
                        if ArticleDate in str(ele):
                            Article_Year = ele[0].text
                            year_col.append(Article_Year)
                            
                        if AuthorList in str(ele):
                            for author in ele:
                                if len(author) >= 2:
                                    Lname = author[0].text
                                    Iname = author[1].text
                                    complName = Lname + ', ' + Iname
                                else:
                                    Lname = author[0].text
                                    complName = Lname
                                Author_List.append(complName)
                        
                if KeywordList in str(item):
                    for ele in item:
                        keyword = ele.text
                        keyword_list.append(keyword)
    
    Authors_col.append(Author_List)
    Keyword_col.append(keyword_list)
    
    #Getting Info from the GEO website
    print("Getting Info from the GEO website")
    URL_GEO_web = "https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=" + GEOacc
    print(URL_GEO_web) 
    
    response2 = requests.get(URL_GEO_web)
    my_file = response2.content.decode()

    
    #Organism Key Word:
    mouse1 = 'Mus musculus'
    human1 = 'Homo sapiens'

    if mouse1 in my_file:
        is_mouse = 'yes'
    else:
        is_mouse = 'no'
    Mouse_col.append(is_mouse)
            
    if human1 in my_file:
        is_human = 'yes'
    else:
        is_human = 'no'
    Human_col.append(is_human)
    

03-Dec-2018 12:08:30 DEBUG utils - Directory ./ already exists. Skipping.
03-Dec-2018 12:08:30 INFO GEOparse - File already exist: using local version.
03-Dec-2018 12:08:30 INFO GEOparse - Parsing ./GSE98119_family.soft.gz: 
03-Dec-2018 12:08:30 DEBUG GEOparse - DATABASE: GeoMiame
03-Dec-2018 12:08:30 DEBUG GEOparse - SERIES: GSE98119
03-Dec-2018 12:08:30 DEBUG GEOparse - PLATFORM: GPL13112
03-Dec-2018 12:08:30 DEBUG GEOparse - PLATFORM: GPL17021
03-Dec-2018 12:08:30 DEBUG GEOparse - PLATFORM: GPL18573
03-Dec-2018 12:08:30 DEBUG GEOparse - PLATFORM: GPL19057
03-Dec-2018 12:08:30 DEBUG GEOparse - PLATFORM: GPL21273
03-Dec-2018 12:08:30 DEBUG GEOparse - PLATFORM: GPL21493
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587372
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587373
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587375
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587376
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587378
03-Dec-2018 12:08:30 DEBUG GEOparse 

GSE98119
Get the GEO information from GEO accession


03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587396
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587397
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587398
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587399
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587400
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587401
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587402
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587403
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587404
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587405
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587406
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587407
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587408
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587409
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587411
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587412
03-Dec-2018 12:08:30 DEBUG GEOparse - SAMPLE: GSM2587414
03-Dec-2018 12:08:30 DEBUG GEOp

03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139012
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139013
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139014
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139015
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139016
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139017
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139018
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139019
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139020
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139021
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139022
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139023
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139024
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139025
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139026
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139027
03-Dec-2018 12:08:31 DEBUG GEOparse - SAMPLE: GSM3139028
03-Dec-2018 12:08:31 DEBUG GEOp

03-Dec-2018 12:08:32 DEBUG GEOparse - SAMPLE: GSM3139156
03-Dec-2018 12:08:32 DEBUG GEOparse - SAMPLE: GSM3139157
03-Dec-2018 12:08:32 DEBUG GEOparse - SAMPLE: GSM3139158
03-Dec-2018 12:08:32 DEBUG GEOparse - SAMPLE: GSM3139159
03-Dec-2018 12:08:32 DEBUG GEOparse - SAMPLE: GSM3139160
03-Dec-2018 12:08:32 DEBUG GEOparse - SAMPLE: GSM3139161
03-Dec-2018 12:08:33 DEBUG GEOparse - SAMPLE: GSM3139162
03-Dec-2018 12:08:33 DEBUG GEOparse - SAMPLE: GSM3139163
03-Dec-2018 12:08:33 DEBUG GEOparse - SAMPLE: GSM3139164
03-Dec-2018 12:08:33 DEBUG GEOparse - SAMPLE: GSM3139165
03-Dec-2018 12:08:33 DEBUG GEOparse - SAMPLE: GSM3139166
03-Dec-2018 12:08:33 DEBUG GEOparse - SAMPLE: GSM3139167
03-Dec-2018 12:08:33 DEBUG GEOparse - SAMPLE: GSM3139168
03-Dec-2018 12:08:33 DEBUG GEOparse - SAMPLE: GSM3139169
03-Dec-2018 12:08:33 DEBUG GEOparse - SAMPLE: GSM3139170
03-Dec-2018 12:08:33 DEBUG GEOparse - SAMPLE: GSM3139171
03-Dec-2018 12:08:33 DEBUG GEOparse - SAMPLE: GSM3139172
03-Dec-2018 12:08:33 DEBUG GEOp

...

Getting Info from the GEO website
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE96033


03-Dec-2018 12:10:39 DEBUG utils - Directory ./ already exists. Skipping.
03-Dec-2018 12:10:39 INFO GEOparse - File already exist: using local version.
03-Dec-2018 12:10:39 INFO GEOparse - Parsing ./GSE117207_family.soft.gz: 
03-Dec-2018 12:10:39 DEBUG GEOparse - DATABASE: GeoMiame
03-Dec-2018 12:10:39 DEBUG GEOparse - SERIES: GSE117207
03-Dec-2018 12:10:39 DEBUG GEOparse - PLATFORM: GPL17021
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM3273537
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM3273538
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM3273539
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM3273540
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM3273541
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM3273542
03-Dec-2018 12:10:39 DEBUG utils - Directory ./ already exists. Skipping.
03-Dec-2018 12:10:39 INFO GEOparse - File already exist: using local version.
03-Dec-2018 12:10:39 INFO GEOparse - Parsing ./GSE109671_family.soft.gz: 
03-Dec-2018 12:10:39 DEBUG GEOparse 

GSE117207
Get the GEO information from GEO accession
GSE109671
Get the GEO information from GEO accession


03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948220
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948221
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948222
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948223
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948224
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948225
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948226
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948227
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948228
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948229
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948230
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948231
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948232
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948233
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948234
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948235
03-Dec-2018 12:10:39 DEBUG GEOparse - SAMPLE: GSM2948236
03-Dec-2018 12:10:39 DEBUG GEOp

ParseError: not well-formed (invalid token): line 1, column 0 (<string>)

In [ ]:
#Make sure the lists the same number of elements

In [ ]:
print(len(GEO_list))
print(len(pubID_col))
print(len(Authors_col))
#Technique_col = []
print(len(Human_col))
print(len(Mouse_col))
print(len(Keyword_col))
#print(len(year_col))
print(len(pubmed_errors_list))
print(len(parseGEO_error_list))

In [ ]:
#Check if publications in the list are in the portal

In [ ]:
# get key from file
my_key = get_key('default')
# all usable env names
all_envs = ['data', 'staging', 'fourfront-webdev', 'fourfront-mastertest', 'fourfront-hotseat']
# get admin key from s3
#my_key = ff_utils.get_authentication_with_server({}, ff_env = 'fourfront-hotseat')

In [ ]:
q = '/search/?type=Publication'
results = ff_utils.search_metadata(q, key=my_key)

In [ ]:
res_title_list=[]
res_uuid_list =[]
for i in results:
    res_title_list.append(i['title'])
    res_uuid_list.append(i['uuid'])

In [ ]:
print(len(res_title_list))
print(len(res_uuid_list))

In [ ]:
print(res_title_list)

In [ ]:
#Get the index of the publications that are already in the portal
index_list = []
for title in Title_col:
    if title in res_title_list:
        x = -1
        for ele in res_title_list:
            x = x + 1
            if title == ele:
                status = 'submitted'
                url = 'https://data.4dnucleome.org/publications/' + res_uuid_list[x]
                break
                   
    else:
        status = 'no_submitted'
        url = 'N/A'
    status_col.append(status)      
    link_col.append(url)
    
        

In [ ]:
COLUMN = ['GEO-Accession','PMID','Title','Authors','Human','Mouse','Keywords', 'Score', 'Status','Link to Portal']
Dic = {'GEO-Accession':GEO_list,'PMID':pubID_col,'Title':Title_col,'Authors':Authors_col,'Human':Human_col,'Mouse':Mouse_col,'Keywords':Keyword_col,'Score':scores_list,'Status':status_col,'Link to Portal':link_col}

In [ ]:
new_df = pandas.DataFrame(Dic, columns=COLUMN)
print(new_df['Status'])

In [ ]:
new_df.to_excel('results.xlsx')

In [ ]:
print(len(link_col))

In [ ]:
print(status_col)

In [ ]:
print(link_col)

In [ ]:
print(Title_col)

In [ ]:
print(res_title_list)

In [ ]:
# for i in Authors_col:
#     str_Authors_col = ','.join(str(e) for e in Authors_col)
# for i in Keyword_col: 
#     str_Keyword_col = ','.join(str(e) for e in Keyword_col)
    

In [9]:
#Get the GEO information from GEO accession
gse = GEOparse.get_GEO('GSE71831')

03-Dec-2018 12:14:03 DEBUG utils - Directory ./ already exists. Skipping.
03-Dec-2018 12:14:03 INFO GEOparse - File already exist: using local version.
03-Dec-2018 12:14:03 INFO GEOparse - Parsing ./GSE71831_family.soft.gz: 
03-Dec-2018 12:14:03 DEBUG GEOparse - DATABASE: GeoMiame
03-Dec-2018 12:14:03 DEBUG GEOparse - SERIES: GSE71831
03-Dec-2018 12:14:03 DEBUG GEOparse - PLATFORM: GPL11154
03-Dec-2018 12:14:03 DEBUG GEOparse - PLATFORM: GPL16791
03-Dec-2018 12:14:03 DEBUG GEOparse - PLATFORM: GPL17021
03-Dec-2018 12:14:03 DEBUG GEOparse - PLATFORM: GPL19129
03-Dec-2018 12:14:03 DEBUG GEOparse - PLATFORM: GPL20795
03-Dec-2018 12:14:03 DEBUG GEOparse - SAMPLE: GSM1847520
03-Dec-2018 12:14:03 DEBUG GEOparse - SAMPLE: GSM1847521
03-Dec-2018 12:14:03 DEBUG GEOparse - SAMPLE: GSM1847522
03-Dec-2018 12:14:03 DEBUG GEOparse - SAMPLE: GSM1847523
03-Dec-2018 12:14:03 DEBUG GEOparse - SAMPLE: GSM1847524
03-Dec-2018 12:14:03 DEBUG GEOparse - SAMPLE: GSM1847525
03-Dec-2018 12:14:03 DEBUG GEOparse 

In [ ]:
#Get metadata
#metadata = gse.metadata
metadata.keys()

In [10]:
#Get Data of interest
pubID_i = gse.metadata['pubmed_id']
contributors = gse.metadata['contributor']
summary = gse.metadata['summary']
exp_design = gse.metadata['overall_design']
exp_type = gse.metadata['type']

In [11]:
#Getting infor from pubmed using the pubmed ID
pubID = str(pubID_i[0])
#Getting xml format
pubmed_URL_xml = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=" + pubID +"&retmode=xml"
print(pubmed_URL_xml)
response = requests.get(pubmed_URL_xml)
# with open('file.xml', 'wb') as f:
#     f.write(response.content)
tree = ET.fromstring(response.content)


https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=27432957&retmode=xml


In [13]:
#The Data is stored in the child of the root
# child = tree.getchildren()
# print(child)
#root = tree.getroot()
for i in tree:
    child = i
print(child)


<Element 'PubmedArticle' at 0x11155b5e8>


In [14]:
#The data is stored in the child of the root
#The info of interest is in the medlineCitation level
medlineCitation = 'MedlineCitation'
#This are the elements of interest in this datalevel:
Article = 'Article'
KeywordList = 'KeywordList' #each element only has a single item. no index needed with for loop


In [15]:
#Elements of Interest in Article level:
AuthorList = 'AuthorList'  # there are several, each item has last name index = 0, first name = 1, and abbrev = 2
ArticleTitle = 'ArticleTitle' # there is only one object so index = 0
ArticleDate = 'ArticleDate' # there are several, each item has a Year index = 0, Month index = 0, and day index = 3

In [16]:
for i in child:
    if medlineCitation in str(i):
        for item in i:
            print(item)
            if Article in str(item):
                for ele in item:
                    print(ele)
                    if ArticleTitle in str(ele):
                        Article_Title = ele.text
                        print(Article_Title)
                        
                    if ArticleDate in str(ele):
                        Article_Year = ele[0].text
                        print(Article_Year)
                            
                    if AuthorList in str(ele):
                        Author_List = []
                        for author in ele:
                            Lname = author[0].text
                            Iname = author[1].text
                            complName = Lname + ', ' + Iname
                            Author_List.append(complName)
                        print(Author_List)
                        
            
            if KeywordList in str(item):
                keyword_list = []
                for ele in item:
                    keyword = ele.text
                    keyword_list.append(keyword)
                print(keyword_list)
    #Getting Info from the GEO website
    URL_GEO_web = "https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=" + 'GSE71831'
    print(URL_GEO_web)
    
    response2 = requests.get(URL_GEO_web)
    my_file = response2.content.decode()
    #Organism Key Word:
    mouse1 = 'Mus musculus'
    human1 = 'Homo sapiens'

    if mouse1 in my_file:
        is_mouse = 'yes'
        
    if human1 in my_file:
        print('human')
        is_human = 'yes'
    


<Element 'PMID' at 0x11155b3b8>
<Element 'DateCompleted' at 0x11155b688>
<Element 'DateRevised' at 0x11155b188>
<Element 'Article' at 0x11154def8>
<Element 'Journal' at 0x11154de58>
<Element 'ArticleTitle' at 0x11154d8b8>
Deletion of DXZ4 on the human inactive X chromosome alters higher-order genome architecture.
<Element 'Pagination' at 0x11154db38>
<Element 'ELocationID' at 0x11154d9f8>
<Element 'Abstract' at 0x11154d818>
<Element 'AuthorList' at 0x11154d7c8>
['Darrow, Emily M', 'Huntley, Miriam H', 'Dudchenko, Olga', 'Stamenova, Elena K', 'Durand, Neva C', 'Sun, Zhuo', 'Huang, Su-Chen', 'Sanborn, Adrian L', 'Machol, Ido', 'Shamim, Muhammad', 'Seberg, Andrew P', 'Lander, Eric S', 'Chadwick, Brian P', 'Aiden, Erez Lieberman']
<Element 'Language' at 0x11152e728>
<Element 'GrantList' at 0x11152e5e8>
<Element 'PublicationTypeList' at 0x111523cc8>
<Element 'ArticleDate' at 0x111523ae8>
2016
<Element 'MedlineJournalInfo' at 0x111523908>
<Element 'ChemicalList' at 0x1115232c8>
<Element 'Cit

In [17]:
Author_List

['Darrow, Emily M',
 'Huntley, Miriam H',
 'Dudchenko, Olga',
 'Stamenova, Elena K',
 'Durand, Neva C',
 'Sun, Zhuo',
 'Huang, Su-Chen',
 'Sanborn, Adrian L',
 'Machol, Ido',
 'Shamim, Muhammad',
 'Seberg, Andrew P',
 'Lander, Eric S',
 'Chadwick, Brian P',
 'Aiden, Erez Lieberman']

In [18]:
Article_Year

'2016'

In [19]:
Article_Title


'Deletion of DXZ4 on the human inactive X chromosome alters higher-order genome architecture.'

In [20]:
keyword_list

['CTCF',
 'Hi‐C',
 'X chromosome inactivation',
 'genome engineering',
 'inactive X chromosome']

In [ ]:
#Getting Info from the GEO website
URL_GEO_web = "https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=" + GEOacc
print(URL_GEO_web)

In [ ]:
response2 = requests.get(URL_GEO_web)
my_file = response2.content.decode()


In [ ]:
#Organism Key Word:
mouse1 = 'Mus musculus'
human1 = 'Homo sapiens'

if mouse1 in my_file:
    print('mouse')
    is_mouse = 'yes'
if human1 in my_file:
    print('human')
    is_human = 'yes'

In [ ]:
#New Table
pubID_col = []
Title_col = []
Authors_col = []
Technique_col = []
Human_col = []
Mouse_col = []
Keyword_col = []
year_col = []
score_col = []

new_df = pandas.DataFrame({'pubID':pubID_col, 'title':Title_col}, columns=['pubID', 'title'])